In [1]:
import re
from collections import Counter
from nltk.tokenize import word_tokenize, sent_tokenize
import torch
from torch.utils.data import DataLoader
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import Laplace
nltk.download('punkt')

class PromptDataset(torch.utils.data.Dataset):
    def __init__(self, text_path, prompt_length):
        self.prompts = self.load_sentences(text_path, prompt_length)
        
    def load_sentences(self, text_path, prompt_length=2):
        prompts = []
        with open(text_path, 'r') as f:
            text = f.read()
        # Split text into sentences
        sentences = sent_tokenize(text.strip())
        for sent in sentences[1:201]:
            tokenized_sent = word_tokenize(sent.lower())
            remove_punct = [word for word in tokenized_sent if re.search(r'\w+', word) is not None]
            prompt = ' '.join(remove_punct[:prompt_length])
            real = ' '.join(remove_punct)
            prompts.append((prompt, real))
        return prompts
    
    def __len__(self):
        return len(self.prompts)
    
    def __getitem__(self, idx):
        return self.prompts[idx]


def count_repetition_percent(generated_strings):
    words = generated_strings.split()
    counter = Counter(words)
    count_repeated = 0
    for word in counter:
        count_repeated += (counter[word] - 1)
    sentence_length = sum(counter.values())
    return count_repeated / sentence_length

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
with open("train_Sherlock.txt", "r") as f:
    text = f.read()
# text = text.replace("\n", " ").replace(u'\ufeff', '')
text = re.sub(r"ADVENTURE.{0,100}\n", "", text)
text = re.sub(r'[^\w\s.]', '', text)
text = re.sub(r"\n+", " ", text)

In [ ]:
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(text)]

In [ ]:
train_data, padded_sents = padded_everygram_pipeline(3, tokenized_text)

In [ ]:
model = Laplace(3)

In [ ]:
len(model.vocab)

0

In [ ]:
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 7993 items>


In [ ]:
count_dict = {}
for word in model.vocab.__iter__():
    count_dict[word] = model.counts[word]

In [ ]:
from heapq import nlargest

nlargest(50, count_dict, key=count_dict.get)

['<s>',
 '</s>',
 '.',
 'the',
 'and',
 'i',
 'of',
 'to',
 'a',
 'in',
 'that',
 'it',
 'he',
 'you',
 'was',
 'his',
 'is',
 'my',
 'have',
 'as',
 'with',
 'had',
 'which',
 'at',
 'for',
 'not',
 'but',
 'me',
 'be',
 'we',
 'from',
 'this',
 'said',
 'there',
 'upon',
 'holmes',
 'him',
 'so',
 'she',
 'her',
 'very',
 'been',
 'your',
 'what',
 'no',
 'all',
 'on',
 'one',
 'then',
 'were']

In [ ]:
from heapq import nlargest
from random import choices


def beam_search(model, vocab, current_sentences_score, ngram_order, beam_size=1):
    candidate_sentences_with_score = {}

    if len(current_sentences_score) == 0:
        for word in vocab:
            candidate_sentences_with_score[word] = model.logscore(word)

    for sent in current_sentences_score.keys():
        if sent.split()[-1] == "</s>":
            candidate_sentences_with_score[sent] = current_sentences_score[sent]
            continue

        cur_length = len(sent.split())
        if cur_length < ngram_order:
            for word in vocab:
                candidate_sentences_with_score[f"{sent} {word}"] = current_sentences_score[sent] + model.logscore(
                    word, sent.split())
        else:
            for word in vocab:
                candidate_sentences_with_score[f"{sent} {word}"] = current_sentences_score[sent] + model.logscore(
                    word, sent.split()[-ngram_order+1:])

    best_sentences = nlargest(
        beam_size, candidate_sentences_with_score, key=candidate_sentences_with_score.get)
    return {sent: candidate_sentences_with_score[sent] for sent in best_sentences}


def sampling(model, vocab, current_sentence, ngram_order, top_k=None, top_p=None):
    vocab_score = {}
    if len(current_sentence) == 0:
        for word in vocab:
            vocab_score[word] = model.score(word)

    else:
        sent_split = current_sentence.split()
        if sent_split[-1] == "</s>":
            return current_sentence

        if len(sent_split) < ngram_order:
            for word in vocab:
                vocab_score[word] = model.score(word, sent_split)
        else:
            for word in vocab:
                vocab_score[word] = model.score(
                    word, sent_split[-ngram_order+1:])

    if top_k:
        top_k_score = {}
        top_k_words = nlargest(top_k, vocab_score, vocab_score.get)
        for word in top_k_words:
            top_k_score[word] = vocab_score[word]
        current_sentence += " {}".format(
            choices(list(top_k_score.keys()), weights=list(top_k_score.values()))[0])

    elif top_p:
        sorted_word_score = sorted(
            vocab_score.items(), key=lambda x: x[1], reverse=True)
        top_p_score = {}
        curr_p = 0
        for word, score in sorted_word_score:
            if curr_p > top_p:
                break
            top_p_score[word] = score
            curr_p += score

        current_sentence += " {}".format(
            choices(list(top_p_score.keys()), weights=list(top_p_score.values()))[0])

    else:
        current_sentence += " {}".format(
            choices(
                list(vocab_score.keys()),
                weights=list(vocab_score.values())
            )[0]
        )
    return current_sentence


def generate_sent(model, mode, num_words=20, init_sent="<s>", beam_size=3, top_k=None, top_p=None):
    assert mode in ["greedy", "beam", "sampling", "nucleus"]
    vocab = list(model.vocab.__iter__())
    vocab.remove("<s>")

    if mode == "greedy":
        current_sentences_score = {init_sent: 0}
        for i in range(num_words):
            current_sentences_score = beam_search(
                model, vocab, current_sentences_score, model.order, beam_size=1)
        return current_sentences_score

    if mode == "beam":
        current_sentences_score = {init_sent: 0}
        for i in range(num_words):
            current_sentences_score = beam_search(
                model, vocab, current_sentences_score, model.order, beam_size=beam_size)
        return current_sentences_score

    if mode == "sampling":
        current_sentence = init_sent
        for i in range(num_words):
            current_sentence = sampling(
                model, vocab, current_sentence, model.order, top_k, top_p)
        return current_sentence


In [ ]:
init_sent = "<s> what"

print("Greedy: \n{}\n".format(generate_sent(model, "beam", init_sent=init_sent, beam_size=1)))
print("Beam: \n{}\n".format(generate_sent(model, "beam", init_sent=init_sent, beam_size=3)))
print("Random sampling: \n{}\n".format(generate_sent(model, "sampling", init_sent=init_sent)))
print("Top k sampling: \n{}\n".format(generate_sent(model, "sampling", init_sent=init_sent, top_k=100)))
print("Nucleus sampling: \n{}\n".format(generate_sent(model, "sampling", init_sent=init_sent, top_p=0.9)))

Greedy: 
{'<s> what do you think that i have been a very serious one to the other side . </s>': -38.550964165354756}

Beam: 
{'<s> what could it be i could see that you will excuse my saying so something just a baby her wee hand': -27.823510507283483, '<s> what could it be i could see that you will excuse my saying so something just a little more . </s>': -30.87033136403648, '<s> what could it be i could see that you will excuse my saying so something just a little too much .': -32.38490453686624}

Random sampling: 
<s> what a blind fool i have spoiled him . </s>

Top k sampling: 
<s> what did you observe anything very funny cried our client has rather an early appointment this morning had hurried up the

Nucleus sampling: 
<s> what can it mean i gasped . </s>



In [ ]:
test_data = PromptDataset('./test_Sherlock.txt', 2)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

In [ ]:
generated_dict = {
    'greedy': [],
    'beam': [],
    'random': [],
    'top_k': [],
    'top_p_95': [],
    'top_p_90': []
}

for i, prompt in enumerate(test_loader):
    context = prompt[0][0]

    greedy = list(generate_sent(model, "beam", init_sent=context, beam_size=1).keys())[0]
    generated_dict['greedy'].append(greedy)

    beam = list(generate_sent(model, "beam", init_sent=context, beam_size=5).keys())[0]
    generated_dict['beam'].append(beam)

    random = generate_sent(model, "sampling", init_sent=context)
    generated_dict['random'].append(random)

    top_k = generate_sent(model, "sampling", init_sent=context, top_k=40)
    generated_dict['top_k'].append(top_k)

    top_p_95 = generate_sent(model, "sampling", init_sent=context, top_p=0.95)
    generated_dict['top_p_95'].append(top_p_95)

    top_p_90 = generate_sent(model, "sampling", init_sent=context, top_p=0.90)
    generated_dict['top_p_90'].append(top_p_90)

    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [ ]:
rep_percent = {
    'greedy': None,
    'beam': None,
    'random': None,
    'top_k': None,
    'top_p_95': None,
    'top_p_90': None
}

for type in generated_dict.keys():
    sum_percent = 0
    for sent in generated_dict[type]:
        sent = re.sub(r'</s>', '', sent)
        sent = re.sub(r'[.]', '', sent)
        sum_percent += count_repetition_percent(sent)
    rep_percent[type] = sum_percent / len(generated_dict[type])

In [ ]:
print(rep_percent)

{'greedy': 0.28457900147238424, 'beam': 0.2691777597402601, 'random': 0.2940398232873082, 'top_k': 0.19146011992393588, 'top_p_95': 0.2974515826021249, 'top_p_90': 0.3018054517721705}
